In [1]:
from datasets import Dataset
import pandas as pd
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer, set_seed
import os
from sklearn.model_selection import train_test_split
from scipy.special import softmax
import argparse
import logging
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import wandb
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import os
from torch.utils.data import Dataset
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from datasets import Dataset
import numpy as np
import numpy as np
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix

import pandas as pd,os
import torch
from statistics import mode
from sklearn.metrics import confusion_matrix
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, Trainer
from datasets import Dataset, load_metric
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
import pickle

c:\Users\Ghiki\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def preprocess_function(examples, **fn_kwargs):
    return fn_kwargs['tokenizer'](examples["text"], truncation=True)

def get_data(train_path, test_path, random_seed):
    """
    function to read dataframe with columns
    """

    train_df = pd.read_json(train_path, lines=True)
    test_df = pd.read_json(test_path, lines=True)
   
    return train_df, test_df

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    auc = roc_auc_score(labels, preds)
    cm = confusion_matrix(labels, preds)

    return {
        'accuracy': acc,
        'f1': f1,
        'auc': auc,
        'precision': precision,
        'recall': recall,
        'confusion_matrix': cm.tolist()
    }

def fine_tune(train_df, valid_df, checkpoints_path, id2label, label2id, model):

    # pandas dataframe to huggingface Dataset
    train_dataset = Dataset.from_pandas(train_df)
    valid_dataset = Dataset.from_pandas(valid_df)
    
    # get tokenizer and model from huggingface
    tokenizer = AutoTokenizer.from_pretrained(model)     # put your model here
    model = AutoModelForSequenceClassification.from_pretrained(
       model, num_labels=len(label2id), id2label=id2label, label2id=label2id    # put your model here
    )
    
    # tokenize data for train/valid
    tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True, fn_kwargs={'tokenizer': tokenizer})
    tokenized_valid_dataset = valid_dataset.map(preprocess_function, batched=True,  fn_kwargs={'tokenizer': tokenizer})
    

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


    # create Trainer 
    training_args = TrainingArguments(
        output_dir=checkpoints_path,
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_valid_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    
    pokemonRojoFuego=trainer.predict(tokenized_valid_dataset)
    print(pokemonRojoFuego)

    # save best model
    best_model_path = checkpoints_path+'/best/'
    
    if not os.path.exists(best_model_path):
        os.makedirs(best_model_path)
    

    trainer.save_model(best_model_path)

def test(test_df, model_path, id2label, label2id):
    
    # load tokenizer from saved model 
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # load best model
    model = AutoModelForSequenceClassification.from_pretrained(
       model_path, num_labels=len(label2id), id2label=id2label, label2id=label2id
    )
            
    test_dataset = Dataset.from_pandas(test_df)

    tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True,  fn_kwargs={'tokenizer': tokenizer})
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # create Trainer
    trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    # get logits from predictions and evaluate results using classification report
    predictions = trainer.predict(tokenized_test_dataset)
    prob_pred = softmax(predictions.predictions, axis=-1)
    preds = np.argmax(predictions.predictions, axis=-1)
    metric = evaluate.load("bstrai/classification_report")
    results = metric.compute(predictions=preds, references=predictions.label_ids)
    
    # return dictionary of classification report
    return results, preds

def process_and_balance_dataframes(df):
    if 'label' not in df.columns:
        raise ValueError("DataFrame must contain a 'label' column")

    unique_labels = df['label'].unique()
    balanced_dfs = []

    for label in np.sort(unique_labels):
        
        # Create a binary label DataFrame
        binary_df = df.copy()
        binary_df['label'] = binary_df['label'].apply(lambda x: 1 if x == label else 0)

        # Balancing the dataset
        rus = RandomUnderSampler(random_state=42)
        X = binary_df.drop('label', axis=1)
        y = binary_df['label']
        X_res, y_res = rus.fit_resample(X, y)

        # Creating a balanced DataFrame
        balanced_df = pd.DataFrame(X_res, columns=X.columns)
        balanced_df['label'] = y_res
        balanced_dfs.append(balanced_df)
        print(f'label: {label} - {balanced_df.shape}')

    return balanced_dfs

In [3]:
random_seed = 0

df= pd.read_json('datasets/subtaskB_train.jsonl', lines=True)
df=df.sample(1000)

print(df['label'].value_counts())

onevsall_dfs=process_and_balance_dataframes(df)

i=0

for current_df in onevsall_dfs:
    print(f'\n\nTraining for label {i}')
    print(current_df['label'].value_counts())
    
    train_df, dev_df = train_test_split(current_df, test_size=0.1, random_state=42)

    train_df.to_json('reducedTrainDataFrame.jsonl', orient='records', lines=True)
    dev_df.to_json('reducedTrainDataFrame_dev.jsonl', orient='records', lines=True)

    train_path =  'reducedTrainDataFrame.jsonl'
    test_path =  'reducedTrainDataFrame_dev.jsonl'

    model = 'roberta-base'

    subtask =  'A'
    prediction_path = 'reducedPredictedDataFrame.jsonl'

    if not os.path.exists(train_path):
        logging.error("File doesnt exists: {}".format(train_path))
        raise ValueError("File doesnt exists: {}".format(train_path))

    if not os.path.exists(test_path):
        logging.error("File doesnt exists: {}".format(train_path))
        raise ValueError("File doesnt exists: {}".format(train_path))

    id2label = {0: "not_current_model", 1: "current_model"}
    label2id = {"not_current_model": 0, "current_model": 1}

    set_seed(random_seed)

    train_df, test_df = get_data(train_path, test_path, random_seed)

    fine_tune(train_df, test_df, f"testing_models/{model.split('/')[-1]}_label{i}_0k", id2label, label2id, model)

    i+=1


label
3    178
4    174
1    172
2    169
0    159
5    148
Name: count, dtype: int64
label: 0 - (318, 5)
label: 1 - (344, 5)
label: 2 - (338, 5)
label: 3 - (356, 5)
label: 4 - (348, 5)
label: 5 - (296, 5)


Training for label 0
label
0    159
1    159
Name: count, dtype: int64


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 32/32 [00:00<00:00, 2282.54 examples/s]
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alberto-rodero557. Use `wandb login --relogin` to force relogin


 33%|███▎      | 18/54 [00:04<00:07,  4.65it/s]Trainer is attempting to log a value of "[[11, 5], [1, 15]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
                                               
 33%|███▎      | 18/54 [00:04<00:07,  4.65it/s]

{'eval_loss': 0.5817766189575195, 'eval_accuracy': 0.8125, 'eval_f1': 0.8333333333333334, 'eval_auc': 0.8125, 'eval_precision': 0.75, 'eval_recall': 0.9375, 'eval_confusion_matrix': [[11, 5], [1, 15]], 'eval_runtime': 0.1643, 'eval_samples_per_second': 194.738, 'eval_steps_per_second': 12.171, 'epoch': 1.0}


 67%|██████▋   | 36/54 [00:10<00:03,  4.65it/s]Trainer is attempting to log a value of "[[16, 0], [2, 14]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
                                               
 67%|██████▋   | 36/54 [00:10<00:03,  4.65it/s]

...

{'eval_loss': 0.21711711585521698, 'eval_accuracy': 0.9333333333333333, 'eval_f1': 0.9444444444444444, 'eval_auc': 0.9230769230769231, 'eval_precision': 0.8947368421052632, 'eval_recall': 1.0, 'eval_confusion_matrix': [[11, 2], [0, 17]], 'eval_runtime': 0.1549, 'eval_samples_per_second': 193.685, 'eval_steps_per_second': 12.912, 'epoch': 3.0}


100%|██████████| 51/51 [00:15<00:00,  3.25it/s]


{'train_runtime': 15.697, 'train_samples_per_second': 50.838, 'train_steps_per_second': 3.249, 'train_loss': 0.4747910219080308, 'epoch': 3.0}


100%|██████████| 2/2 [00:00<00:00, 30.05it/s]


PredictionOutput(predictions=array([[-1.1500847 ,  1.1474271 ],
       [-0.7840526 ,  0.6739131 ],
       [ 1.8492463 , -1.9941918 ],
       [ 1.813197  , -1.9462798 ],
       [-0.8954138 ,  0.77748954],
       [ 1.7625248 , -1.8589479 ],
       [ 1.6846746 , -1.9103608 ],
       [-1.2534664 ,  1.3212765 ],
       [-1.0698038 ,  1.0213008 ],
       [-0.9071823 ,  0.8377258 ],
       [-1.0452782 ,  1.0803397 ],
       [-1.0825732 ,  1.1000346 ],
       [ 1.7938638 , -1.825214  ],
       [-1.1138895 ,  1.1410542 ],
       [-1.2842449 ,  1.287307  ],
       [ 1.6619502 , -1.5945144 ],
       [-0.9776328 ,  0.87743306],
       [-1.1470191 ,  1.2442986 ],
       [-0.92999357,  0.9439759 ],
       [ 1.7240598 , -1.9179192 ],
       [ 1.7239888 , -1.7073314 ],
       [-0.67987186,  0.7178457 ],
       [-1.2051665 ,  1.1256298 ],
       [ 1.753485  , -1.8150482 ],
       [ 1.8350328 , -1.8648995 ],
       [ 1.221591  , -1.1349193 ],
       [-0.86628574,  0.94437146],
       [-0.9059453 ,  1.05